In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn
import random
import collections
import time

In [2]:
training_file = 'story.txt'
tf.reset_default_graph()

In [3]:
start_time = time.time()
def elapsed(sec):
    if sec<60:
        return str(sec) + "sec"
    elif sec<(60 * 60):
        return str(sec/60) + "min"
    else:
        return str(sec/(60 * 60)) + "hr"

In [4]:
#path declre
logs_path = r"C:\Users\PABITRA\SELF"
writter = tf.summary.FileWriter(logs_path)

In [5]:
#read data

def read_data(fname):
    with open(fname) as f:
        content = f.readlines()
    content = [x.strip() for x in content]
    content = [content[i].split() for i in range(len(content))]
    content = np.array(content)
    content = np.reshape(content, [-1, ])
    return content

In [6]:
training_data = read_data(training_file)
print("loaded training data....")

loaded training data....


In [7]:
#frequency of occurance word into a dictionary
def build_dataset(words):
    count = collections.Counter(words).most_common()
    dictionary = dict()
    for word,_ in count:
        dictionary[word] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return dictionary, reverse_dictionary

In [8]:
dictionary, reverse_dictionary = build_dataset(training_data)
vocab_size = len(dictionary)
vocab_size

113

In [9]:
#declare model parametrs

learning_rate = 0.001
training_iters = 50000
display_step = 1000
n_input = 3
n_hidden = 512

In [10]:
#DECLARE MODEL PLACEHOLDER
 
x = tf.placeholder("float", [None, n_input, 1])
y = tf.placeholder("float", [None, vocab_size])

#DECLARE MODEL VARIABLES

weights = { 'out': tf.Variable(tf.random_normal([n_hidden, vocab_size]))}
biases = { 'out': tf.Variable(tf.random_normal([vocab_size]))}

#DECLARE MODEL OPERATION

def RNN(x, weights, biases):
    #reshape x into[1, n_input]
    x= tf.reshape(x, [-1, n_input])
    
    #generate a n_input element sequence of inputs
    #eg [had] [a] [ general]
    x = tf.split(x, n_input,1)
    
    #2-layer lstm with 512 hidden units
    rnn_cell = rnn.MultiRNNCell([rnn.BasicLSTMCell(n_hidden), rnn.BasicLSTMCell(n_hidden)])
    
    #generate prediction
    outputs,states = rnn.static_rnn(rnn_cell, x, dtype=tf.float32)
         
    
    #there are n_input outputs but we only want the last output
    return tf.add(tf.matmul(outputs[-1], weights['out']), biases['out'])

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
#prediction
pred = RNN(x, weights, biases)

#declare loss
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels=y))

#declare optimizer
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [12]:
#MODEL EVALUATION
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [13]:

#variable initializer
init = tf.global_variables_initializer()

In [ ]:
# Launch the graph
with tf.Session() as session:
    session.run(init)
    step = 0
    offset = random.randint(0,n_input+1)
    end_offset = n_input + 1
    acc_total = 0
    loss_total = 0
 
    #writer.add_graph(session.graph)
 
    while step < training_iters: 
        offset = random.randint(0, n_input+1)
 
        symbols_in_keys = [ [dictionary[ str(training_data[i])]] for i in range(offset, offset+n_input) ]
        symbols_in_keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
 
        symbols_out_onehot = np.zeros([vocab_size], dtype=float)
        symbols_out_onehot[dictionary[str(training_data[offset+n_input])]] = 1.0
        symbols_out_onehot = np.reshape(symbols_out_onehot,[1,-1])
 
        _, acc, loss, onehot_pred = session.run([optimizer, accuracy, cost, pred], 
                                                feed_dict={x: symbols_in_keys, y: symbols_out_onehot})
        loss_total += loss
        acc_total += acc
        if (step+1) % display_step == 0:
            print("Iter= " + str(step+1) + ", Average Loss= " +
                  "{:.6f}".format(loss_total/display_step) + ", Average Accuracy= " +
                  "{:.2f}%".format(100*acc_total/display_step))
            acc_total = 0
            loss_total = 0
            symbols_in = [training_data[i] for i in range(offset, offset + n_input)]
            symbols_out = training_data[offset + n_input]
            symbols_out_pred = reverse_dictionary[int(tf.argmax(onehot_pred, 1).eval())]
            print("%s - [%s] vs [%s]" % (symbols_in,symbols_out,symbols_out_pred))
        step += 1
        offset += (n_input+1)
    print("Optimization Finished!")
    print("Elapsed time: ", elapsed(time.time() - start_time))
    print("Run on command line.")
    print(" tensorboard --logdir=%s" % (logs_path))
    print("Point your web browser to: http://localhost:6006/")
    while True:
        prompt = "%s words: " % n_input
        sentence = input(prompt)
        sentence = sentence.strip()
        words = sentence.split(' ')
        if len(words) != n_input:
            continue
        try:
            symbols_in_keys = [dictionary[str(words[i])] for i in range(len(words))]
            for i in range(32):
                keys = np.reshape(np.array(symbols_in_keys), [-1, n_input, 1])
                onehot_pred = session.run(pred, feed_dict={x: keys})
                onehot_pred_index = int(tf.argmax(onehot_pred, 1).eval())
                sentence = "%s %s" % (sentence,reverse_dictionary[onehot_pred_index])
                symbols_in_keys = symbols_in_keys[1:]
                symbols_in_keys.append(onehot_pred_index)
            print(sentence)
        except:
            print("Word not in dictionary")

Iter= 1000, Average Loss= 0.455655, Average Accuracy= 90.50%
['long', 'ago,', 'the'] - [mice] vs [mice]
Iter= 2000, Average Loss= 0.000000, Average Accuracy= 100.00%
['the', 'mice', 'had'] - [a] vs [a]
Iter= 3000, Average Loss= 0.000000, Average Accuracy= 100.00%
['ago,', 'the', 'mice'] - [had] vs [had]
Iter= 4000, Average Loss= 0.000000, Average Accuracy= 100.00%
['mice', 'had', 'a'] - [general] vs [general]
Iter= 5000, Average Loss= 0.000000, Average Accuracy= 100.00%
['had', 'a', 'general'] - [council] vs [council]
Iter= 6000, Average Loss= 0.000000, Average Accuracy= 100.00%
['had', 'a', 'general'] - [council] vs [council]
Iter= 7000, Average Loss= 0.000000, Average Accuracy= 100.00%
['long', 'ago,', 'the'] - [mice] vs [mice]
Iter= 8000, Average Loss= 0.000000, Average Accuracy= 100.00%
['ago,', 'the', 'mice'] - [had] vs [had]
Iter= 9000, Average Loss= 0.000000, Average Accuracy= 100.00%
['ago,', 'the', 'mice'] - [had] vs [had]
Iter= 10000, Average Loss= 0.000000, Average Accuracy=